https://www.worldfootball.net/report/premier-league-2015-2016-manchester-united-tottenham-hotspur/

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import itertools

In [2]:
import lxml

In [3]:
import requests

In [4]:
source_team = requests.get('https://www.worldfootball.net/players/eng-premier-league-2015-2016/').text

In [5]:
soup_team = BeautifulSoup(source_team,'lxml')

In [6]:
table_teams = soup_team.find_all(class_='standard_tabelle')[0]

In [7]:
fixtures_columns = ['home_team','away_team']
for which_team in ['home', 'away']:
    for i in range(1,12):
        fixtures_columns.append(which_team +(str(i)))
fixtures_columns.append('home_score')
fixtures_columns.append('away_score')

In [8]:
fixtures = pd.DataFrame(columns=fixtures_columns)

In [9]:
fixtures

,home_team,away_team,home1,home2,home3,home4,home5,home6,home7,home8,...,away4,away5,away6,away7,away8,away9,away10,away11,home_score,away_score


In [33]:
players_columns = ['name','team','season']

In [37]:
players_df = pd.DataFrame(columns=players_columns)

In [12]:
teams_list = []
for i in table_teams.contents:
    if len(i) > 1:
        teams_list.append(i.contents[1].contents[1]['href'].split('teams/')[1].split('/')[0])

In [13]:
teams_list

['afc-bournemouth',
 'arsenal-fc',
 'aston-villa',
 'chelsea-fc',
 'crystal-palace',
 'everton-fc',
 'leicester-city',
 'liverpool-fc',
 'manchester-city',
 'manchester-united',
 'newcastle-united',
 'norwich-city',
 'southampton-fc',
 'stoke-city',
 'sunderland-afc',
 'swansea-city',
 'tottenham-hotspur',
 'watford-fc',
 'west-bromwich-albion',
 'west-ham-united']

In [21]:
every_fixture = []

for team_1 in teams_list:
    for team_2 in teams_list:
        if team_1 != team_2:
            every_fixture.append([team_1,team_2])

In [22]:
every_fixture[20]

['arsenal-fc', 'aston-villa']

In [124]:
def sarting_lineup(table_players,players_list):
    counter = 0
    for i in table_players.contents:
        if (len(i) > 4) & (counter <11):
            players_list.append(i.contents[3].contents[1]['title'])
            counter +=1
    
def scrape_match(season,team_home,team_away):
    html = 'https://www.worldfootball.net/report/premier-league-' + season + '-' + team_home + '-' +team_away
    print html
    source = requests.get(html).text
    soup = BeautifulSoup(source,'lxml')
    score = soup.find_all(class_='standard_tabelle')[0]
    table_players_home = soup.find_all(class_='standard_tabelle')[2]
    if len(table_players_home.contents) > 10:
        table_players_away = soup.find_all(class_='standard_tabelle')[3]
    else:
        table_players_home = soup.find_all(class_='standard_tabelle')[3]
        table_players_away = soup.find_all(class_='standard_tabelle')[4]
    players_list = []
    sarting_lineup(table_players_home,players_list)
    sarting_lineup(table_players_away,players_list)
    text=score.contents[3].contents[3].contents[1].text
    score_arr = []
    for c in text: 
        if c.isdigit():
            score_arr.append(int(c))
    return players_list,score_arr

In [77]:
players_list,score_arr = scrape_match('2015-2016',every_fixture[20][0],every_fixture[20][1])

In [68]:
players_list

[u'Petr \xc4\x8cech',
 ' Gabriel Paulista',
 'Laurent Koscielny',
 ' Nacho Monreal',
 u' H\xc3\xa9ctor Beller\xc3\xadn',
 'Jack Wilshere',
 u'Mesut \xc3\x96zil',
 ' Santi Cazorla',
 'Francis Coquelin',
 'Olivier Giroud',
 u'Alexis S\xc3\xa1nchez',
 'Mark Bunn',
 'Leandro Bacuna',
 'Joleon Lescott',
 'Aly Cissokho',
 'Kevin Toner',
 'Idrissa Gueye',
 'Scott Sinclair',
 'Ashley Westwood',
 u'Carlos S\xc3\xa1nchez',
 'Jordan Lyden',
 'Jordan Ayew']

In [69]:
score_arr

[4, 0]

In [70]:
fixtures.append(pd.Series(every_fixture[20] +players_list +score_arr,index=fixtures.columns),ignore_index=True)

,home_team,away_team,home1,home2,home3,home4,home5,home6,home7,home8,...,away4,away5,away6,away7,away8,away9,away10,away11,home_score,away_score
0,arsenal-fc,aston-villa,Petr Äech,Gabriel Paulista,Laurent Koscielny,Nacho Monreal,HÃ©ctor BellerÃ­n,Jack Wilshere,Mesut Ãzil,Santi Cazorla,...,Aly Cissokho,Kevin Toner,Idrissa Gueye,Scott Sinclair,Ashley Westwood,Carlos SÃ¡nchez,Jordan Lyden,Jordan Ayew,4.0,0.0


In [72]:
players_df = pd.DataFrame(columns=players_columns)

In [74]:
players_df_current = pd.DataFrame(columns=players_columns)
players_df_current['name'] = players_list
players_df_current['team'] = [every_fixture[20][0]]*11+[every_fixture[20][1]]*11
players_df_current['season'] = "2016"

In [75]:
players_df_current

,name,team,season
0,Petr Äech,arsenal-fc,2016
1,Gabriel Paulista,arsenal-fc,2016
2,Laurent Koscielny,arsenal-fc,2016
3,Nacho Monreal,arsenal-fc,2016
4,HÃ©ctor BellerÃ­n,arsenal-fc,2016
5,Jack Wilshere,arsenal-fc,2016
6,Mesut Ãzil,arsenal-fc,2016
7,Santi Cazorla,arsenal-fc,2016
8,Francis Coquelin,arsenal-fc,2016
9,Olivier Giroud,arsenal-fc,2016


In [134]:
def scrape_season(season):
    seaon_source = str(season - 1) + '-' + str(season)
    source_team = requests.get('https://www.worldfootball.net/players/eng-premier-league-'+seaon_source +'/').text
    soup_team = BeautifulSoup(source_team,'lxml')
    table_teams = soup_team.find_all(class_='standard_tabelle')[0]
    teams_list = []
    for i in table_teams.contents:
        if len(i) > 1:
            teams_list.append(i.contents[1].contents[1]['href'].split('teams/')[1].split('/')[0])
    fixtures_columns = ['home_team','away_team']
    for which_team in ['home', 'away']:
        for i in range(1,12):
            fixtures_columns.append(which_team +(str(i)))
    fixtures_columns.append('home_score')
    fixtures_columns.append('away_score')
    fixtures = pd.DataFrame(columns=fixtures_columns)
    players_columns = ['name','team']
    players_df = pd.DataFrame(columns=players_columns)
    every_fixture = []
    for team_1 in teams_list:
        for team_2 in teams_list:
            if team_1 != team_2:
                every_fixture.append([team_1,team_2])
    for fixture in every_fixture:
        players_list,score_arr = scrape_match(seaon_source,fixture[0],fixture[1])
        fixtures = fixtures.append(pd.Series(fixture +players_list +score_arr,index=fixtures.columns),ignore_index=True)
        players_df_current = pd.DataFrame(columns=players_columns)
        players_df_current['name'] = players_list
        players_df_current['team'] = [fixture[0]]*11+[fixture[1]]*11
        players_df = pd.concat([players_df,players_df_current])
        print fixtures
        print players_df
    
    fixtures['season'] = str(season)
    players_df['season'] = str(season)
    return fixtures,players_df

In [ ]:
fixtures_2016,players_df_2016 = scrape_season(2016)

https://www.worldfootball.net/report/premier-league-2015-2016-afc-bournemouth-arsenal-fc
         home_team   away_team        home1          home2       home3  \
0  afc-bournemouth  arsenal-fc  Artur Boruc  Simon Francis  Steve Cook   

             home4       home5        home6          home7      home8  \
0  Charlie Daniels  Adam Smith  Dan Gosling  Andrew Surman  Marc Pugh   

     ...               away4               away5        away6         away7  \
0    ...       Nacho Monreal   HÃ©ctor BellerÃ­n  Mesut Ãzil  Aaron Ramsey   

             away8           away9                   away10           away11  \
0  Mathieu Flamini  Olivier Giroud  Alex Oxlade-Chamberlain  Alexis SÃ¡nchez   

  home_score away_score  
0        0.0        2.0  

[1 rows x 26 columns]
                       name             team
0               Artur Boruc  afc-bournemouth
1             Simon Francis  afc-bournemouth
2                Steve Cook  afc-bournemouth
3           Charlie Daniels  afc-bournem

In [125]:
players_list,score_arr = scrape_match('2015-2016',every_fixture[359][0],every_fixture[359][1])

https://www.worldfootball.net/report/premier-league-2015-2016-west-bromwich-albion-watford-fc


In [126]:
players_list

['Ben Foster',
 'James Chester',
 'Jonny Evans',
 'Gareth McAuley',
 'Craig Dawson',
 'Claudio Yacob',
 'Darren Fletcher',
 u'St\xc3\xa9phane Sess\xc3\xa8gnon',
 'James McClean',
 'Saido Berahino',
 u'Salom\xc3\xb3n Rond\xc3\xb3n',
 ' Gomes',
 'Allan Nyom',
 'Miguel Britos',
 u'Sebastian Pr\xc3\xb6dl',
 u'Nathan Ak\xc3\xa9',
 ' Jurado',
 u'Adl\xc3\xa8ne Gu\xc3\xa9dioura',
 'Ben Watson',
 u'\xc3\x89tienne Capoue',
 'Troy Deeney',
 'Odion Ighalo']

In [122]:
html = 'https://www.worldfootball.net/report/premier-league-' + '2015-2016' + '-' + every_fixture[359][0] + '-' +every_fixture[359][1]
source = requests.get(html).text
soup = BeautifulSoup(source,'lxml')
score = soup.find_all(class_='standard_tabelle')[0]
table_players_home = soup.find_all(class_='standard_tabelle')[2]
table_players_away = soup.find_all(class_='standard_tabelle')[3]

In [123]:
len(table_players_home.contents)

7

In [128]:
fixtures_2016

,home_team,away_team,home1,home2,home3,home4,home5,home6,home7,home8,...,away5,away6,away7,away8,away9,away10,away11,home_score,away_score,season


In [129]:
players_df

,name,team,season
